In [1]:
from core import langda_solve

In [2]:
model = """
# Try langda solve:
operation(X,Y,Z) :-
    langda(LLM:"X plus Y equals Z",FUP:"false").
"""
additional_input = { "config": { "configurable": { "thread_id": "42" } },"prefix": "","langda_ext": ""}

In [3]:
result = langda_solve('simple',model,'deepseek-chat',additional_input=additional_input)

print(result)

### =========== processing init_node =========== ###
{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}
[{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}]
processing _decide_next_init ...
No langda needs to be updated, process end now...
### =========== processing summary_node =========== ###
*** test_result: ***

Running problog_test_tool...
Error evaluating Problog model:
    toks = self.label_tokens(string, root_tokens)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zhenzhili/miniforge3/envs/langda/lib/python3.11/site-packages/problog/parser.py", line 1186, in label_tokens
    raise ParseError(string, "Expected binary operator", t.location)
problog.parser.ParseError: Expected binary operator at 1:2.
{'C5731F2D': 'operation(X, Y, Z) :- Z is X + Y.'}
*** Running_time: 0s, 0 rounds in total.
# Try langda solve:
operation(X,Y,Z) :-
 
 Z is X + Y.


In [4]:
print(result)

# Try langda solve:
operation(X,Y,Z) :-
 
 Z is X + Y.
